# Unsupervised learning models

Unsupervised learning is area of machine learning focused on detecting patterns in the data and **modelling without explicitly set labels/target variable**. In contrast, supervised learning techniques are mainly based on predicting nominal features (classification) or continuous features (regression).

Main tasks in the area of unsupervised learning are:
- **dimensionality reduction**
- **clustering**
- anomaly detection

**Dimensionality reduction** algorithms aim to represent high-dimensional input data in the output space with lower dimensionality. The approach is useful for:
- visualization of high dimensional data
- removing noise
- lowering the volume of the dataset, hence improving performance of other algorithms
- obfuscating and anonymizing the data

**Clustering** aims to differentiate the groups within the data, usually based on the distance between the observations. It's common task for customer or product datasets - segments created based on clustering results may be used in marketing activities or as an input to supervised machine learning model.

Importing required libraries:

In [ ]:
using MultivariateStats, RDatasets, Plots, StatsBase, LinearAlgebra, ClusterAnalysis
using UMAP: umap
using Statistics

## Dimensionality reduction

Loading a dataset:

In [ ]:
data = dataset("datasets", "USArrests")

Transforming a dataframe into numerical matrix structure, similarly to X matrix for predictive models composed of exogenous variables:

In [ ]:
X_matrix = Matrix(data[:, 2:end])

Keeping US names in a seperate `labels` variable:

In [ ]:
labels = Vector(data[:, 1])

Saving data dimensions to `obs_number` and `original_size` variables:

In [ ]:
obs_number, original_size = size(X_matrix)

## Principal Component Analysis (PCA)

We'll use PCA (Principal Components Analysis) technique to represent high dimensional data in 2-dimensional space and plot the result.

PCA is popular algorithm for dimensionality reduction based on linear algebra. For input matrix (dataset) we need to calculate eigenvectors (principal components) and eigenvalues. Eigenvectors determine directions for projection in new feature space and eigenvalues determine the mangnitude ('importance') of the vectors.

We start with an quick-and-easy simulated data set to generate visualisations which helps to explain the concept of PCA. Later, we move on to the real dataset.

In [ ]:
corr = 0.9
n=1000
corr_mat = [1 corr;
            corr 1]
chol_decomp = cholesky(corr_mat).L
cor_data=[randn(n) randn(n)]*chol_decomp'
println("Correlation matrix: ", cor(cor_data))
println("Standard deviations: ", std(cor_data, dims=1))
print("Means: ", mean(cor_data, dims=1))


In [ ]:
fitted_PCA = fit(PCA, cor_data', maxoutdim = 2, mean=0)

In [ ]:
scatter(cor_data[:, 1], cor_data[:, 2], markeralpha=0.05, markercolor=:lightslateblue, rightmargin=5Plots.mm,
        ylim=(-4, 4), xlim=(-4, 4), aspect_ratio=:equal, markersize=2, layout=(1, 2),
        xlabel="X1", ylabel="X2", legend=false, title="Input correlated \ndata for PCA")
quiver!(zeros(2), zeros(2),
        quiver=(loadings(fitted_PCA)[1, :], loadings(fitted_PCA)[2, :]), linewidth=2,
        c=:black)
scatter!(predict(fitted_PCA, cor_data')'[:, 1], predict(fitted_PCA, cor_data')'[:, 2],
        markeralpha=0.05, markercolor=:lightslateblue, subplot=2,
        ylim=(-4, 4), xlim=(-4, 4), aspect_ratio=:equal, markersize=2,
        xlabel="1st Principal Component", ylabel="2nd Principal Component",
        legend=false, title="Output orthogonal \nPrincipal Components")
quiver!(zeros(2), zeros(2), subplot=2,
        quiver=(predict(fitted_PCA, loadings(fitted_PCA))'[1, :],
                predict(fitted_PCA, loadings(fitted_PCA))'[2, :]),
        linewidth=2, c=:black)

PCA requires a data standarization to work properly. PCA on a non-standarized data will be biased towards variables with highest variance.

In [ ]:
standarized_X_matrix = Matrix(mapcols(zscore, data[:,2:end]))

Fitting PCA on a transposed `standarized_X_matrix` with means equal to $0$:

In [ ]:
fitted_PCA = fit(PCA, standarized_X_matrix';
                maxoutdim = original_size, mean=0)

#### Questions:
- What's the interpretation of loadings?
- Why loadings are unstandarized? what it means to be standarized here?
- What's the interpretation of Eigenvalues?
- Explain the 'Importance of components' table in a business language.                

In [ ]:
standarized_loadings = LinearAlgebra.eigvecs(fitted_PCA)[:,1:2]

In [ ]:
standarized_loadings = LinearAlgebra.eigvecs(fitted_PCA)[:,1:2]

Now you can predict Principal Components (also knows as embedding). You can think of Principal Components as new features characterizing observations. Usually you'll neeed signficanlty less number of Principal Compenents than original number of columns to represent the significant share of information (variance) repsented in an original dataset.

In [ ]:
PrincipalComponents = predict(fitted_PCA, standarized_X_matrix')

Both `predict` and `MultivariateStats.transform` will produce Pricipal Components - you can think of those as encoding the original features:

In [ ]:
PrincipalComponents == MultivariateStats.transform(fitted_PCA, standarized_X_matrix') #encoding

In [ ]:
PC_limit = ceil(Int, maximum(abs.(PrincipalComponents)))

In [ ]:
UStates_visualization = [ (PrincipalComponents'[i,1], PrincipalComponents'[i,2],
                        text(data.State[i], 6, :blue)) for i=1:obs_number ]

In [ ]:
loadings_visualization = [ text(names(data)[i], 10, :orange, :right) for i=1:original_size ]

So-called **biplot** is the important visualization tool of PCA. Biplot dipsplays both the **pricipal componenets score** (observations, e.g. states) and the **principal component loadings**, i.e. coefficients/weights defining each of Principal Components and enabling the interpretation of Pricipal Components also in a business langaguage. Below is the **biplot** for our PCA:

In [ ]:
# Replication of FIGURE 12.1 on p. 502 from https://hastie.su.domains/ISLR2/ISLRv2_website.pdf
plot( ; annotations= UStates_visualization , legend = false,
    xlim=[-PC_limit-.5,PC_limit+.5], ylim=[-PC_limit, PC_limit],
    xlabel = "1st Principal Component",
    ylabel = "2nd Principal Component")
hline!([0],line=:dash, color=:grey)
vline!([0],line=:dash, color=:grey)
quiver!(zeros(original_size),
        zeros(original_size),
        quiver=(standarized_loadings[:,1],standarized_loadings[:,2]),
        c=:orange)
annotate!(standarized_loadings[:,1]*1.1, 1.1*standarized_loadings[:,2], loadings_visualization)

#### Questions:
- Is the plot the same as in FIGURE 12.1?
- If not, why is so? is it a problem?

How efficient is reduction? How much information, i.e. variance, is "encoded" in low-dimentional embedding?

In [ ]:
explained_variance = LinearAlgebra.eigvals(fitted_PCA)

Both `LinearAlgebra.eigvals()` and `principalvars()` generate sequence of decreasing variance explained by corresponding Principal Component:

In [ ]:
explained_variance == principalvars(fitted_PCA)

Why does `explained_variance` sum to the number of features? 

In [ ]:
sum(explained_variance)

It's more convenient also in business terms to talk about the proportion of explained Variance (the same way we prefer to speak about $R^2$ - coefficient of determination rather than Mean Squared Error) rather than absolute value of total variance being explained, so we calculate `propOfExplainedVariance`: 

In [ ]:
propOfExplainedVariance = explained_variance ./ sum(explained_variance)

Below is the replication of **FIGURE 12.3** from page 507 from [**An Introduction to Statistical Learning**](https://hastie.su.domains/ISLR2/ISLRv2_website.pdf):
- on the left: so called  **scree plot** depicting the proportion of variance explained by each of prciipal components
- on the right: the cumulative proportion of variance explained by principal components.

In [ ]:
plot(1:length(explained_variance), 
    Matrix( [propOfExplainedVariance  cumsum(propOfExplainedVariance)]),
    ylim = [0,1],marker=:circle, layout=(1,2),legend = false,
    xlabel = "Principal Component", rightmargin=5Plots.mm,
    ylabel = ["Prop. Variance Explained" "Cumulative Prop. Variance Explained"])

You might want to use your reduced number of Principal Components to predict `original_size` features and see how good your PCA is able to recover original variance. This process is called "decoding":

In [ ]:
recovered_original_space_obs = reconstruct(fit(PCA, standarized_X_matrix';
                                                maxoutdim = 2, mean=0),
                                            PrincipalComponents[1:2,:])'

Let's visualize how well 2-dimensional PCA is able to reconstruct 4-dimensional original set:

In [ ]:
gr(display_type=:inline) # to adjust bottom margin in order not to overlap with titles of lower row of plot
scatter(recovered_original_space_obs,
        standarized_X_matrix,
        layout = (2, 2), rightmargin=5Plots.mm,
        ylabel = "Predicted",
        xlabel = "Original value",
        title=["Murder" "Assault" "UrbanPop" "Rape"],
        legend = false, xlim=(-2,2), ylim=(-2,2))
Plots.abline!(1, 0, line=:dash,  subplot = 1)
Plots.abline!(1, 0, line=:dash,  subplot = 2)
Plots.abline!(1, 0, line=:dash,  subplot = 3)
Plots.abline!(1, 0, line=:dash,  subplot = 4)

PCA exploits the correlation between features, which means that variables bring similar information to each other and this information can be represented (encoded, embedded) in Principal Components. If features were orthogonal (not correlated) to each other, the would be no sense and value-added from using PCA. Luckily, typicaly in observational (in comparison to experimental) socio-economic data there's a lot of correlated variables as you can see bellow in a following correlation matrix:

In [ ]:
cor(standarized_X_matrix)

### Summary of Principal Component Analysis (PCA)
- New representation in lower dimensional space (e.g. less than 10 or 2 for visuall purposes) is achieved via linear combination of original features (usually more than 100) and loadings
- PCA exploits the correlation between features. The would be no sens and no value-added of using PCA, if variables were orthoghonal, i.e. idealy independent.
- PCA is equivalent to deep learning autoencoder with linear activation functions
- PCA requires data standarization, as its ommision will results in relative focus on variables with higher variance
- even though PCA results of Principal Components don't have direct interpretation, they can be thought as indices being composed of original features and being able to represent high dimenstional (often >100) dataset by less than 10 Principal Components 

## Uniform Manifold Approximation & Projection (UMAP)
Now let's try a modern dimensionality reduction algorithm called **UMAP** (Uniform Manifold Approximation & Projection). It is rooted in Riemannian geometry - details can be found in the [paper](https://arxiv.org/abs/1802.03426). UMAP proved to give really good results and is considered state-of-the-art.

Function `umap` returns a new data representation into 2-dimentional embedding. An important input parameter is `n_neighbors`:

In [ ]:
UMAP_embedding = umap(standarized_X_matrix', n_neighbors = 10)'

As previously, we set up a vector composed of 50 states describing its embedding coordinates and graphical parameters:

In [ ]:
UStates_visualization = [ (UMAP_embedding[i,1], UMAP_embedding[i,2],
                        text(data.State[i], 6, :blue)) for i=1:obs_number ]

Finally, we draw a two-dimensional scatterplot with our observations of US states into embedded space: 

In [ ]:
#PC_limit = ceil(Int, maximum(PrincipalComponents))
plot( ; annotations= UStates_visualization , legend = false,
    xlim=[floor(Int, minimum(UMAP_embedding[:,1])), ceil(Int, maximum(UMAP_embedding[:,1]))],
    ylim=[floor(Int, minimum(UMAP_embedding[:,2])), ceil(Int, maximum(UMAP_embedding[:,2]))],
    xlabel = "1st dimension of UMAP embedding",
    ylabel = "2nd dimension of UMAP embedding")


## Clustering
### k-means algorithm

K-means algorithm in the nutshell:
1. Pick randomly 'k' observations from the dataset - initial centroids
2. Assign other observations to the nearest centroid
3. Calculate average coordinates from the members of the clusters - new coordinates of the center
4. Repeat 2. and 3. until stop criterion is reached

We start with a simple and simulated dataset demonstrating the potential Data Generating Process for k-mean algorithm and how it can be applied to this dataset. We start with defining two basic data generation parameters:

In [ ]:
n_per_cluster = 100
true_centroids = [(2, 2), (6, 6), (2,6), (6,2)]


Data Generating Process (DGP):

In [ ]:
observations = (true_centroids[1] .+ randn(2,n_per_cluster))'
for centroid_number in 2:length(true_centroids)
    observations = vcat(observations, (true_centroids[centroid_number] .+ randn(2,n_per_cluster))')
end

Generated data visualization:

In [ ]:
scatter(observations[:,1], observations[:,2], color=repeat(1:length(true_centroids),
        inner=n_per_cluster), xlab="X1", ylab="X2", legend=false, title="Simulated clustered data")

Fitting k-means algorithm:

In [ ]:
fitted_kmeans=kmeans(observations, 4, nstart = 100, maxiter = 100)


Visualization of k-means clustering:

In [ ]:
scatter(observations[:,1], observations[:,2], color=fitted_kmeans.cluster,
        xlabel="X1", ylab="X2", legend = false, title="Data clustered by k-means")
estimated_centroids = reduce(hcat, fitted_kmeans.centroids)
scatter!(estimated_centroids[1,:], estimated_centroids[2,:],markershape=:xcross,
        markersize = 10, markercolor=:black, msw=11)

Generating an artificial dense prediction set to create a classification regeions:

In [ ]:
denseset_n=10000
denseset=[8*rand(denseset_n) rand(denseset_n)*8]
predicted_cluster=[argmin(sum((denseset[i,:] .- estimated_centroids) .^2, dims = 1))[2] for i in 1:denseset_n]

In [ ]:
scatter(denseset[:,1], denseset[:,2], color=predicted_cluster, xlabel="X1", ylab="X2",
        markersize=6, msw=0,title="Classification regions", legend=false)
scatter!(observations[:,1], observations[:,2],
        color=:black, xlabel="X1", ylab="X2", markersize=2)
scatter!(estimated_centroids[1,:], estimated_centroids[2,:],markershape=:xcross,
        markersize = 10, markercolor=:white, msw=11)

Then we move to a real-world dataset:

In [ ]:
fitted_kmeans=kmeans(standarized_X_matrix, 4, nstart = 100, maxiter = 100)

In [ ]:
scatter(PrincipalComponents'[:,1], PrincipalComponents'[:,2], color=fitted_kmeans.cluster, legend = false,
xlabel = "1st Principal Component",
ylabel = "2nd Principal Component",
title = "K-means clustering in PCA embedding")

In [ ]:
scatter(UMAP_embedding[:,1], UMAP_embedding[:,2], color=fitted_kmeans.cluster, legend = false,
xlabel = "1st dimension of UMAP embedding",
ylabel = "2nd dimension of UMAP embedding",
title = "k-means clustering in UMAP embedding")

**Elbow** method to pick k - inertia for given k-means clustering is the sum of squares between clusters' center and their members

In [ ]:
max_number_of_clusters = 20
within_sum_squares=[kmeans(standarized_X_matrix, k, nstart=500, maxiter=100).withinss
                    for k in 1:max_number_of_clusters]
plot(1:max_number_of_clusters, within_sum_squares,
    ylim=(0,within_sum_squares[1]), legend = false,
    xlabel = "Number of clusters",
    ylabel = "Within sum of squares",
    title = "Elbow plot")

### Summary of k-means algorithm
- k-means might a support segmentation exercise, but resulted segments will heavily depend on the subjective selection of input variables and number of segments K. These should be specified rather by a subject matter expert rather than a data scientist. Segmentation is more an art than a science and therfore rather not so easy task.
- requires data standarization, since no standarization will result in clusters defined accros variables with high variables
- each run of k-means algorithm starts with a randomized starting points, i.e. cluster centroids, so the results might differ from run to run
- since k-means is not global optimization method, it might identify local solution, so it's important to allow an algorithm to be rerun multiple times with different starting points, i.e. cluster centroids

*Preparation of this workshop has been supported by the Polish National Agency for Academic Exchange under the Strategic Partnerships programme, grant number BPI/PST/2021/1/00069/U/00001.*

![SGH & NAWA](../logo.png)